In [5]:
import time
import random
import numpy as np
from sklearn.linear_model import LinearRegression

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 1, 1, 1
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 1, 1, 1
Xstd,Ystd = 1, 1

m1, k1 = 0.1, -1.0
m2, k2 = 0.25, 2.0
m3, k3 = 0.5, -1.5
m4, k4 = 1.5, 0.0

d1 = 5
d2 = 3
d3 = 7
d4 = 4

errorA = 2
errorB = 1.5
errorC = 0.5
errorX = 0.75
errorY = 0.6

def jitter(x, scale=1):
    return x + np.random.normal(scale=scale)

def dependent(x, m, c, error=1):
    return jitter(m*x + c, scale=error) # mx +c + error


def linear(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    B = dependent(A, m1, k1, errorB)
    C = dependent(B, m2, k2, errorC)
    for i in range(n):
        A, B, C = next_generation(A)

    return np.array([A,B,C])


def radiating(n=100):
    def next_generation(B):
        B_ = jitter(B, errorB)
        A_ = dependent(B_, m1, k1, errorA)
        C_ = dependent(B_, m2, k2, errorC)
        return A_, B_, C_

    B = np.random.normal(loc=Bm, scale=Bstd) # normal distribution, Bm - mean and Bstd - standard deviation
    A = dependent(B, m1, k1, errorA)
    C = dependent(B, m2, k2, errorC)

    for i in range(n):
        A,B,C = next_generation(B)

    return np.array([A, B, C])


def common_cause(n=100):
    def next_generation(X):
        X = jitter(X, errorX)
        A = dependent(X, m1, k1, errorA)
        B = dependent(X, m2, k2, errorB)
        C = dependent(X, m3, k3, errorC)
        return A, B, C, X

    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    A = dependent(X, m1, k1, errorA)
    B = dependent(X, m2, k2, errorB)
    C = dependent(X, m3, k3, errorC)

    for i in range(n):
        A, B, C, X = next_generation(X)

    return np.array([A,B,C])


def single_difference_cause(n=100):

    def next_generation(A, X):
        X = jitter(X, errorX)
        A = jitter(A, errorA)
        B = jitter(m1*A + m2*X + k1, errorB)
        C = dependent(X, m3, k3, errorC)
        return A, B, C, X

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation

    for i in range(n+1):
        A,B,C,X = next_generation(A,X)

    return np.array([A, B, C])


def double_difference_cause(n=100):
    def next_generation(X,Y):
        X = jitter(X, errorX)
        Y = jitter(Y, errorY)
        A = dependent(X, m1, k1, errorA)
        B = jitter(m2*X+m3*Y+k2, errorB)
        C = dependent(Y, m4, k4, errorC)
        return A, B, C, X, Y

    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    Y = np.random.normal(loc=Ym, scale=Ystd) # normal distribution, Ym - mean and Ystd - standard deviation
    X = random.random()
    Y = random.random()

    for i in range(n+1):
        A, B, C, X, Y = next_generation(X, Y)

    return np.array([A, B, C])

def convergent(n=100):

    def next_generation(A, C):
        B_ = jitter(m1*A + m2*C + k1, errorB)
        A_ = jitter(A, errorA)
        C_ = jitter(C, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    C = np.random.normal(loc=Cm, scale=Cstd) # normal distribution, Cm - mean and Cstd - standard deviation

    B = jitter(m1*A + m2*C + k1, errorB)
    for i in range(n):
        A, B, C = next_generation(A, C)

    return np.array([A, B, C])

def simulations_data(pathway, n=1000):
    random.seed(time.time())
    return np.array([pathway() for i in range(n)])


def regress(X,Y):
    model = LinearRegression()
    mXY = model.fit(X.reshape(-1,1), Y)
    r_sqr = mXY.score(X.reshape(-1,1), Y)
    residual = Y - model.predict(X.reshape(-1, 1))
    return mXY.intercept_, mXY.coef_[0], r_sqr, residual


def get_slope_intercept(model):
    return model._slopt, model._intercept

def compute_regresssion(ABC):
    A,B,C = ABC.transpose()
    RAB = regress(A, B)
    RBC = regress(B, C)
    RAC = regress(A, C)
    corrE = np.corrcoef(np.array([RAB[3], RBC[3]]))
    corrE_BA_C = np.corrcoef(np.array([np.square(RAB[3]), C]))


    #print(RAB[1]*RBC[1]-RAC[1]) ## better to look at distribution of this error..it should come with center as 0
    return {"kAB": RAB[0], "kBC":RBC[0], "kAC":RAC[0],
            "mAB":RAB[1], "mBC":RBC[1], "mAC":RAC[1],
            "r_sqrAB":RAB[2], "r_sqrBC":RBC[2], "r_sqrAC":RAC[2],
            "r_E":corrE[0,1],
            "r_E_BA_C":corrE_BA_C[0,1],
            "n": len(A)}

def compute_correlation(ABC):
    corr = np.corrcoef(ABC.transpose())
    rAB, rBC, rAC = corr[0,1], corr[1,2], corr[0,2]
    #print(rAB**2*rBC**2-rAC**2) ## better to look at distribution of this error..it should come with center as 0
                                ## or see correlation between these two quantities should be 1 and if we regress ,
                                ###       it should have slope 1
    return {"rAB":rAB, "rBC":rBC, "rAC":rAC}


def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        ABC = simulations_data(convergent, samplesize) # you can change pathway here
        r = compute_regresssion(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [15]:
def compute_confidence_interval(r, n):
    def boundary(zeta):
        return ((np.exp(2*zeta))-1)/((np.exp(2*zeta))+1)

    z = 0.5*np.log(((1-r)/(1+r)))
    zetal = z-1.96*np.sqrt(1/(n-3))
    rl = boundary(zetal)
    zetau = z+1.96*np.sqrt(1/(n-3))
    ru = boundary(zetau)
    return rl, ru

def confidence_status(L, U, v):
    l = v.where(v < L, "less")
    l.where(v > U, "more", inplace=True)
    w = l.where((v>= L) & (v <= U), "within")
    return w

def confidence_status_(r, n):
    L, U = compute_confidence_interval(r, n)
    return confidence_status(L, U, r)


def add_confidence_stats(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    L,U = compute_confidence_interval(d.rBC**2, d['n'])
    d['confidence_residual_corr'] = confidence_status(L, U, d.r_E)
    L,U = compute_confidence_interval(d.r_E, d['n'])#???
    d['confidence_corrected_bc_corr'] = confidence_status(L, U, d.r_E_BA_C**2)

def confidence_graphs(d):
    confidence = alt.Chart(d, title=f"Correlation confidence{(d['confidence']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_res_corr = alt.Chart(d, title=f"residual corr {(d['confidence_residual_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_residual_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_corrected_bc_corr = alt.Chart(d, title=f"corrected bc corr {(d['confidence_corrected_bc_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_corrected_bc_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    return alt.vconcat(confidence,confidence_res_corr, confidence_corrected_bc_corr)

add_confidence_stats(d)
confidence_graphs(d)

alt.VConcatChart(...)

In [12]:
def stats_graphs(d):
    slope_histogram = alt.Chart(d).mark_bar().encode(
        x=alt.X('mAB*mBC-mAC:Q', bin=True),
        y='count()').properties(title="slope diff histogram")

    bincount = 100
    ticks = 10
    correlation_graph = alt.Chart(d).mark_bar().encode(
        x=alt.X('rAB2*rBC2-rAC2:Q',bin=True,
                   axis=alt.Axis(
                    tickCount=ticks,
                    grid=False)),
        y='count()').properties(
            title="Correlation")
    residual_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X('r_E:Q', bin=True),
        y='count()').properties(title="Correlation of residuals")
    corrected_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X("r_E_BA_C2-rBC2:Q", bin=True),
        y='count()').properties(
        title="Corrected Correlation")

    return alt.vconcat(slope_histogram,correlation_graph,residual_correlation, corrected_correlation)

stats_graphs(d)

alt.VConcatChart(...)

In [13]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")
alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [14]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,n,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence,confidence_residual_corr,confidence_corrected_bc_corr
0,-0.784596,1.536785,0.834203,0.099762,0.969791,0.020792,0.504280,0.241101,0.005616,0.316942,...,500,0.710127,0.491020,0.074937,0.115967,-0.226445,0.075956,within,within,within
1,-0.722763,1.195323,0.657412,0.100243,0.885913,-0.004780,0.484880,0.213883,0.000300,0.371357,...,500,0.696333,0.462475,-0.017333,0.103407,-0.197781,0.093586,within,within,within
2,-0.899076,1.597540,0.943934,0.104717,0.824704,0.013204,0.523218,0.219325,0.002682,0.340230,...,500,0.723338,0.468321,0.051792,0.112073,-0.218574,0.073157,within,within,within
3,-0.602916,1.823962,1.333959,0.094542,0.950982,-0.005672,0.446773,0.226175,0.000402,0.345227,...,500,0.668411,0.475578,-0.020054,0.100647,-0.194537,0.095579,within,within,within
4,-0.800161,1.763061,1.086809,0.101597,0.974737,0.021868,0.471442,0.267204,0.006142,0.305106,...,500,0.686617,0.516917,0.078374,0.119829,-0.266769,0.077163,within,within,within
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.851254,1.203434,0.560626,0.095407,0.919428,0.005623,0.452665,0.257877,0.000480,0.337557,...,500,0.672804,0.507815,0.021901,0.116252,-0.254905,0.082097,within,within,within
96,-0.783419,1.425935,0.876230,0.101490,0.797193,0.001544,0.512654,0.201003,0.000038,0.361299,...,500,0.715999,0.448334,0.006126,0.103008,-0.200980,0.079363,within,within,within
97,-0.806560,1.426096,0.847115,0.095950,0.810611,-0.013346,0.477390,0.187713,0.002638,0.371921,...,500,0.690934,0.433259,-0.051364,0.086974,-0.186876,0.091124,within,within,within
98,-0.593419,1.752005,1.472249,0.099609,0.719798,-0.002268,0.541730,0.170024,0.000092,0.373656,...,500,0.736023,0.412340,-0.009602,0.092015,-0.168877,0.073967,within,within,within
